In [ ]:
# Run in python 3.7+
# Script for all sensor data

import sys
import arable
from   arable.client   import ArableClient as a
a = a()
import re
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from   datetime        import timedelta
from   datetime        import datetime
import pandas          as     pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from   io              import StringIO
from   sensor_includes import email, password_ccber, tenant_ccber
#import arable_data_dict as dd

In [ ]:
# Datetime object to format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverse dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temp lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

In [ ]:
a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

dt = datetime.now()

In [ ]:
device_ids     = [] # Ex. A00****
device_names   = [] # Ex. Lagoon Ice Plants
device_created = [] # Ex. 2019-10-10T00:00:00

# The devices of interest have Z in the Location Name on the Arable website.
# These Location Names are added to device_names.
# The corresponding Device ID is added to device_ids.
for i in a.devices():
    keys = list(i.keys())
    for j in range(len(keys)):
        if str(keys[j]).startswith('location'): 
            values = list(i.values())
            loc = str(values[j])
            loc_name = (re.split(r':', loc)[2])[2:-2]
            if re.search("Z", loc_name):
                device_ids.append(str(values[j-30]))
                device_names.append(loc_name)
                device_created.append(str(values[j-28])[:-7])
print(device_ids)
print(device_names)
print(device_created)

In [ ]:
dates    = [dt_to_ym(i) for i in device_created]
print(device_ids)

levels = np.tile([6, -4,  3, -3, 2, -8, 7, -5, 6,  -5, 1, -1],
                 int(np.ceil(len(dates)/12)))[:len(dates)]

fig, ax = plt.subplots(figsize=(10, 3))
ax.set(title="Sensor Startup Date")

markerline, stemline, baseline = ax.stem(dates, levels,
                                         linefmt="k-", basefmt="k-",
                                         use_line_collection=True)

plt.setp(markerline, mec="k", mfc="w", zorder=3)
markerline.set_ydata(np.zeros(len(dates)))

vert = np.array(['top', 'bottom'])[(levels > 0).astype(int)]
for d, l, r, va in zip(dates, levels, device_ids, vert):
    ax.annotate(r, xy=(d, l), xytext=(-3, np.sign(l)*3),
                textcoords="offset points", va=va, ha="right")

# Format X axis with 4 month intervals
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%d %b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# Remove Y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.5)
plt.show()

In [ ]:
# Converts each item in device_ids to a list.          
device_ids = list(map(lambda el:[el], device_ids))

In [ ]:
def arable_query(s, b, c, d, e, f, g):
    df = a.query(select = str(s),
               format = str(b),
               devices = c,
               measure = str(d),
               order = str(e),
               end = f,
               start = g,
               limit = 100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

lat = []
lon = []
for i in device_ids:
    df = arable_query('all', 'csv', i, "daily", "time", dt_to_ymd_hms(dt), dt_to_ymd_hms(dt - timedelta(days=1)))
    lat.append(float(str(df['lat'].values.astype(float)).strip('[ ]')))
    lon.append(float(str(df['long'].values.astype(float)).strip('[ ]')))

data_location = {'name': device_ids, 'location': device_names, 'lat': lat, 'lon': lon}
df_location   = pd.DataFrame(data=data_location)


In [ ]:
# Plot sensor locations on state and county maps
import altair as alt
from vega_datasets import data
alt.renderers.enable('notebook')

# Load the data as a dict object
us_10m  = data.us_10m()
# Select the geometries under states under objects, filter for California (id = 6)
us_10m['objects']['states']['geometries']=[item for item in us_10m['objects'] \
      ['states']['geometries'] if item['id'] in [6]]
# Make California topojson data
state = alt.Data(
    values=us_10m, 
    format=alt.TopoDataFormat(feature='states',type='topojson'))
# Make California background
background = alt.Chart(state).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

# Make SB County topojson data
SB_County = alt.Data(url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/CA/Santa%20Barbara.geo.json', format=alt.DataFormat(property='features',type='json'))
# SB County as background
county = alt.Chart(SB_County).mark_geoshape(
    fill='lightblue',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

# Plot sensor locations
points = alt.Chart(df_location).mark_point(
    size=1,
    color='black'
).encode(
    longitude='lon:Q',
    latitude='lat:Q'
)

# Label sensor names
text = points.mark_text(
    align='left',
    baseline='bottom',
    dx=9,
    dy=9,
    fontSize=9,
    tooltip={"content": "encoding"}
).encode(
    text='name:O'
)

ca = background + county + points 
sb = county + points + text
#alt.vconcat(ca, sb)
ca | sb

In [ ]:
# Monthly
m_start    = []
m_start_fn = []
m_end      = []
m_end_fn   = []

# Yearly
y_start    = []
y_start_fn = []
y_end      = []
y_end_fn   = []

for j in range(len(device_created)):
    ms  = []
    msf = []
    me  = []
    mef = []
    
    ys  = []
    ysf = []
    ye  = []
    yef = []
    
    def new_month(new_m):
        append_ymd_hms(ms, new_m)
        append_string(me, new_m)
        append_ym(msf, new_m)  
        append_ym(mef, new_m)
        
    def new_year(new_y):
        append_ymd_hms(ys, new_y)
        append_string(ye, new_y)
        append_ym(ysf, new_y)
        append_ym(yef, new_y)
    
    device_created_ymd = (device_created[j])[:10]
    start              = ymd_to_dt(device_created_ymd)
    
    append_string(ms, start)
    append_string(ys, start)
    append_ym(msf, device_created_dt)
    append_ym(ysf, device_created_dt)
    
    start  = start.replace(day=1)
    
    if start.year == dt.year:
        for i in range(start.month+1, dt.month+1):
            new_month(start.replace(month=i))
    
    elif start.year != dt.year: 
        for i in range(dt.year-start.year):
            for j in range(start.month+1, 13): 
                new_month(start.replace(month=j))
        for i in range(start.year+1, dt.year+1):
            newyr = start.replace(year=i, month=1)
            new_year(newyr)
            for j in range(1, dt.month+1):
                new_month(newyr.replace(month=j))
    
    append_string(me, dt)
    append_string(ye, dt)
    append_ym(mef, dt)
    append_ym(yef, dt)
    
    append_temp_list(ms, m_start)
    append_temp_list(me, m_end)
    append_temp_list(msf, m_start_fn)
    append_temp_list(mef, m_end_fn)
    
    append_temp_list(ys, y_start)
    append_temp_list(ye, y_end)
    append_temp_list(ysf, y_start_fn)
    append_temp_list(yef, y_end_fn)

In [ ]:
# Writes a .csv file for each device.
# Hourly data is separated by month; daily data is separated by year.
# Ex. A000***_hourly_2019-01_2019-02.csv
def write_csv_file(sta, end, i, sta_f, end_f, device):
    for j in range(len(sta)):
        try:
            df = a.query(select = 'all', format = 'csv', devices = device, measure = str(i), order = "time", end = end[j], start = sta[j], limit = 100000000)
            fn = str(device)[2:-2] + '_' + str(i) + '_' + str(sta_f[j]) + '_' + str(end_f[j]) + '.csv'
            print(fn)
            df = StringIO(df)
            df = pd.read_csv(df, sep=',', error_bad_lines=False)
            df = df.drop('location', 1)
            df.to_csv(fn, sep = ",")
        except:
            print('ERROR WRITING FILE')
            continue

In [ ]:
# Writes data to a .csv file
f_input = raw_input('Write data to a .csv file? (y/n) ')
if f_input == 'y':
    print('\n Select type of data: ')
    print(str('{:<2d}'.format(1) + '{:<8}'.format('Daily') + '{:<20}'.format('(for each year)')))
    print(str('{:<2d}'.format(2) + '{:<8}'.format('Hourly') + '{:<20}'.format('(for each month)')))
    d_input = int(raw_input('[1-2]  '))
    
    all_q = raw_input('\n All devices? (y/n) ')
    if all_q == 'y':
        for i in range(len(device_ids)):
            if d_input == 1:
                write_csv_file(y_start[i], y_end[i], "daily",  y_start_fn[i], y_end_fn[i], device_ids[i])
            elif d_input == 2:
                write_csv_file(m_start[i], m_end[i], "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
    
    elif all_q == 'n':
        print('\n Select device: ')
        print('-----------------------------------')
        print(str('{:6}'.format('Number')) + '  ' + '{:<7}'.format('ID') + '  ' + '{:<10}'.format('Location Name'))
        print('-----------------------------------')
        for i in range(len(device_ids)):
            print(str('{:<6d}'.format(i+1)) + '  ' + '{:^7}'.format(str(device_ids[i]).strip("[' ']")) + '  ' + '{:^10}'.format(str(device_names[i]).strip("['Z ']")))
            print('-----------------------------------')
        j = int(raw_input('[1-' + str(len(device_ids)) + ']  '))
        if 1 <= j <= int(len(device_ids)):
            i = j
            if d_input == 1:
                write_csv_file(y_start[i], y_end[i], "daily",  y_start_fn[i], y_end_fn[i], device_ids[i])
            elif d_input == 2:
                write_csv_file(m_start[i], m_end[i], "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
        else:
            print('User input error')
    
    else:
        print('User input error')
elif f_input == 'n':
    print('\n No files written.')
else:
    print('User input error')

In [ ]:
de = arable_query('all', 'csv', device_ids[0], "hourly", "time", (m_end[0])[1], (m_start[0])[1])
de['time'] = pd.to_datetime(de['time'],infer_datetime_format=True)

In [ ]:
# Make temperature charts for specific timeframe
charts = []
for i in range(len(device_names)):
    base = alt.Chart(de, title=str((device_names[i])[2:]) + ' \n ' + str(((m_start[i])[-1])[:10]) + ' -- ' + str((m_end[i])[-1])[:10]
    ).mark_line(
        color='black'
    ).properties(
        width=800,
        height=100
    ).encode(
        x=alt.X('time:T', axis=alt.Axis(title='')),
        y=alt.Y('Tabove:Q', axis=alt.Axis(title='Temperature (' + u'\N{DEGREE SIGN}' + 'C)')))

    s = alt.layer( *[base.mark_line(color='blue').encode(y='Tbelow')], data=de)
    z = alt.layer( *[base.mark_line(color='green').encode(y='Tair')], data=de)
    q = alt.layer( *[base.mark_line(color='lightblue').encode(y='Tdew')], data=de)
    graph = base + s + z + q
    charts.append(graph)
alt.vconcat(*charts)

In [ ]:
df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)

fig, ax = plt.subplots(figsize=(3, 3))
ax = plt.gca()

df.plot(kind='line',x='time',y='Tabove',ax=ax)
df.plot(kind='line',x='time',y='Tbelow', color='red', ax=ax)
df.plot(kind='line',x='time',y='Tair', color='green', ax=ax)
df.plot(kind='line',x='time',y='Tdew', color='blue', ax=ax)
plt.xlabel('')
plt.ylabel('Deg. C')
plt.title(str((device_names[0])[1:]) + ' \n ' + str(((sta_m[0])[0])[:10]) + ' -- ' + str((end_m[0])[0])[:10])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()